# Filesystem e accesso ai servizi dell'OS
  
Da [File and Directory Access](https://docs.python.org/3/library/filesys.html),
  [Generic Operating System Services](https://docs.python.org/3/library/allos.html) e
  [Subprocess management](https://docs.python.org/3/library/subprocess.html).
 
## Path orientati ad oggetti

In [1]:
from pathlib import Path

In [2]:
# relativo e assoluto

(
    Path(), 
    Path().absolute(), 
    Path('/Users/santini/Documents/Activities').relative_to('/Users')
)

(PosixPath('.'),
 PosixPath('/Users/santini/Documents/Activities/Teaching/Courses/pybg/repo/moduli/sysadm'),
 PosixPath('santini/Documents/Activities'))

In [3]:
# overloading di / 

Path('~santini') / 'Documents' / 'Dropbox'

PosixPath('~santini/Documents/Dropbox')

In [4]:
# globbing (ache ricorsivo)

list(Path('/etc').glob('sys*'))

[PosixPath('/etc/syslog.conf'), PosixPath('/etc/syslog.conf~previous')]

In [5]:
list(Path().rglob('*.ipynb'))

[PosixPath('01-StandardLibrary.ipynb'),
 PosixPath('02-Fabric.ipynb'),
 PosixPath('.ipynb_checkpoints/Fabric-checkpoint.ipynb'),
 PosixPath('.ipynb_checkpoints/01-StandardLibrary-checkpoint.ipynb')]

In [6]:
# listing 

list(Path().iterdir())

[PosixPath('docker'),
 PosixPath('.nodel'),
 PosixPath('01-StandardLibrary.ipynb'),
 PosixPath('.ipynb_checkpoints'),
 PosixPath('02-Fabric.ipynb')]

In [7]:
# home e cwd (sono metodi statici)

Path.home(), Path.cwd() 

(PosixPath('/Users/santini'),
 PosixPath('/Users/santini/Documents/Activities/Teaching/Courses/pybg/repo/moduli/sysadm'))

In [8]:
# espansione tilde

Path('~admin').expanduser()

PosixPath('/Users/admin')

In [9]:
# query

(
    Path('/pippo').exists(),
    Path('/Users').is_dir(),
    Path('/Users/santini/.bashrc').is_file(),
    Path('/Users/santini/.bash_profile').is_symlink(),
)

(False, True, True, True)

In [10]:
# parti del nome

p = Path('/dir/file.tar.gz')

p.name, p.stem, p.suffixes, p.suffix

('file.tar.gz', 'file.tar', ['.tar', '.gz'], '.gz')

In [11]:
# rimpiazzare nome e suffisso 

p.with_name('newname.ext'), p.with_suffix('.newsuff')

(PosixPath('/dir/newname.ext'), PosixPath('/dir/file.tar.newsuff'))

In [12]:
# parti

p = Path('/Users/santini/.bash_profile')

p.parts

('/', 'Users', 'santini', '.bash_profile')

In [13]:
# parti della directory 

p.parent, list(p.parents) 

(PosixPath('/Users/santini'),
 [PosixPath('/Users/santini'), PosixPath('/Users'), PosixPath('/')])

In [14]:
# permessi

p = Path('~/.bashrc').expanduser()

p.owner(), p.group(), p.stat()

('santini',
 'staff',
 os.stat_result(st_mode=33216, st_ino=41664083, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=3374, st_atime=1571344159, st_mtime=1557902501, st_ctime=1572032566))

In [15]:
# modificare i permessi 

old_mode = p.stat().st_mode

p.chmod(0o000)
print(oct(p.stat().st_mode))

p.chmod(old_mode)
print(oct(p.stat().st_mode))

0o100000
0o100700


In [16]:
# tutta una serie di manipolazioni

# p.rename, p.replace, p.mkdir, p.link_to…

In [17]:
# scrivere e leggere

p = Path('/tmp/test.txt')
p.write_text("""
Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.
""")

print(p.read_text().strip())

Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.


## FIle e directory temporanee

In [18]:
from tempfile import TemporaryFile, TemporaryDirectory, NamedTemporaryFile

In [19]:
with TemporaryFile() as tmp:
    tmp.write(b'123')
    tmp.seek(0)
    print(tmp.read())

b'123'


In [20]:
with TemporaryDirectory() as tmp:
    (Path(tmp) / 'file.txt').write_text('ciao')

Path(tmp).is_dir()

False

In [21]:
with NamedTemporaryFile() as tmp:
    print(tmp.name)

/var/folders/23/yqhr033n6rl4q23_2fry24j80000gn/T/tmpewszlm_6


## Operazioni ad alto livello su gruppi di file

In [22]:
import shutil

In [23]:
shutil.copytree('docker', 'docker-copy')

list(Path('docker-copy').iterdir())

[PosixPath('docker-copy/build_image.sh'),
 PosixPath('docker-copy/Dockerfile'),
 PosixPath('docker-copy/run_slaves.sh'),
 PosixPath('docker-copy/run_controller.sh')]

In [24]:
shutil.rmtree('docker-copy')

Path('docker-copy').is_dir()

False

In [25]:
# il comodo which 

shutil.which('python')

'/Users/santini/.local/share/dir-venv/ee639860a61f45b0491eb86d7b9bb81c-moduli/bin/python'

## Eseguire altri programmi

In [26]:
import subprocess

In [27]:
subprocess.run(['ls', '-l'])

CompletedProcess(args=['ls', '-l'], returncode=0)

In [28]:
# catturare l'output (come testo)

result = subprocess.run(['ls', '-l'], capture_output = True, text = True)
print(result.stdout)

total 44
-rw-r--r-- 1 santini staff 20384 Oct 25 21:42 01-StandardLibrary.ipynb
-rw-r--r-- 1 santini staff 23736 Oct 25 16:43 02-Fabric.ipynb
drwxr-xr-x 6 santini staff   192 Oct 25 20:42 docker



In [29]:
# fare il parsing del comando come fa la shell

print(subprocess.run('ls -l', shell = True, capture_output = True, text = True).stdout)

total 44
-rw-r--r-- 1 santini staff 20384 Oct 25 21:42 01-StandardLibrary.ipynb
-rw-r--r-- 1 santini staff 23736 Oct 25 16:43 02-Fabric.ipynb
drwxr-xr-x 6 santini staff   192 Oct 25 20:42 docker



In [30]:
angiolieri = """S'i fosse fuoco, arderei 'l mondo;
s'i fosse vento, lo tempestarei;
s'i fosse acqua, i' l'annegherei;
""".replace("'", ' ')

subprocess.run('wc -lw', shell = True, input = angiolieri, text = True, capture_output = True).stdout

'      3      20\n'

# Aspetti di rete

Da [Internet Data Handling](https://docs.python.org/3/library/netdata.html), 
 e [Internet Protocols and Support](https://docs.python.org/3/library/internet.html).
 
## Mandare email

In [32]:
# per prova…

# lanciare 'python -m smtpd -c DebuggingServer -n localhost:1025' da shell…

import smtplib

with smtplib.SMTP('localhost', 1025) as server:
    server.sendmail('from@me.info', ['to@you.info'], 'Subject: Test\n\nHello dear')

In [33]:
# Un messaggio html (con fallback testuale)

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# testo

text = """\
Ciao, 
    quarda che bellissimo sito http://homes.di.unimi.it/santini/pybg/ 
credo che possa interessarti!

A presto,
    Massimo
"""
part1 = MIMEText(text, 'plain')

# html

html = """\
<html>
  <body>
    <p>
        Ciao,<br/>
        quarda che <strong>belissimo</strong> sito <a href="http://homes.di.unimi.it/santini/pybg/">http://homes.di.unimi.it/santini/pybg/</a> credo che possa interessarti!
    </p>
    <p>
        A presto,<br/>
        Massimo
    </p>
  </body>
</html>
"""
part2 = MIMEText(html, 'html')

# preparo il messaggio

sender_email = 'massimo.santini@gmail.com'
receiver_email = 'massimo.santini@gmail.com'

message = MIMEMultipart('alternative')
message['Subject'] = 'multipart test'
message['From'] = sender_email
message['To'] = receiver_email

message.attach(part1)
message.attach(part2)

In [34]:
# invio il messaggio

from os import environ
import ssl

context = ssl.create_default_context()
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context = context) as server:
    server.login(sender_email, environ['GMAIL_PW'])
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )

## Lavorare con gli IP

In [36]:
import socket

socket.gethostbyname('homes.di.unimi.it')

'159.149.130.139'

In [38]:
import ipaddress

addr = ipaddress.ip_address(socket.gethostbyname('homes.di.unimi.it'))
addr

IPv4Address('159.149.130.139')

In [44]:
addr.reverse_pointer

'139.130.149.159.in-addr.arpa'

In [47]:
netw = ipaddress.ip_network('159.149.130.128/28')
netw

IPv4Network('159.149.130.128/28')

In [56]:
netw.network_address, netw.broadcast_address

(IPv4Address('159.149.130.128'), IPv4Address('159.149.130.143'))

In [58]:
netw.hostmask, netw.netmask

(IPv4Address('0.0.0.15'), IPv4Address('255.255.255.240'))

In [60]:
addr in netw

True

In [61]:
netw.num_addresses, list(netw.hosts())

(16,
 [IPv4Address('159.149.130.129'),
  IPv4Address('159.149.130.130'),
  IPv4Address('159.149.130.131'),
  IPv4Address('159.149.130.132'),
  IPv4Address('159.149.130.133'),
  IPv4Address('159.149.130.134'),
  IPv4Address('159.149.130.135'),
  IPv4Address('159.149.130.136'),
  IPv4Address('159.149.130.137'),
  IPv4Address('159.149.130.138'),
  IPv4Address('159.149.130.139'),
  IPv4Address('159.149.130.140'),
  IPv4Address('159.149.130.141'),
  IPv4Address('159.149.130.142')])

    
# Gestione del software Python

Da [Creating Standalone Applications with zipapp](https://docs.python.org/3/library/zipapp.html#creating-standalone-applications-with-zipapp), usando [click](https://click.palletsprojects.com/).

In [6]:
! rm -rf greet* && mkdir greet

In [7]:
%%writefile greet/greeter.py

import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name',
              help='The person to greet.')
def hello(count, name):
    for x in range(count):
        click.echo(f'Hello {name}!')

Writing greet/greeter.py


In [8]:
%%writefile greet/__main__.py

from greeter import hello

hello()

Writing greet/__main__.py


In [9]:
! pip install click --target greet && rm -rf greet/*.dist-info

  Using cached https://files.pythonhosted.org/packages/fa/37/45185cb5abbc30d7257104c434fe0b07e5a195a6847506c074527aa599ec/Click-7.0-py2.py3-none-any.whl


In [12]:
! ls greet

__main__.py  click  greeter.py


In [10]:
! python -m zipapp -p '/usr/bin/env python3' greet

In [11]:
! ./greet.pyz --name Massimo --count 3

Hello Massimo!
Hello Massimo!
Hello Massimo!
